<a href="https://colab.research.google.com/github/Kevthiago/Artificial-Intelligence/blob/main/QueryDoct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q google_generativeai

In [3]:
# Import the Python SDK
import google.generativeai as genai
import numpy as np
import pandas as pd

from google.colab import userdata
minha_key = userdata.get("SECRET_API")
genai.configure(api_key= minha_key)

In [4]:
for m in genai.list_models():
  if "embedContent" in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [ ]:
#Exemplo de embedding
title = "Quem não se apegar a IA ficará para trás!"
sample_text = ("Título: Quem não se apegar a IA ficará para trás!"
                "/n"
                "Artigo completo: xxx"
                "\n"
                "Fim da matéria")
embeddings = genai.embed_content(model= "models/embedding-001", content= sample_text, title= title, task_type= "RETRIEVAL_DOCUMENT")
print(embeddings)

In [9]:
#Listagem de documentos que serão buscados

DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema..."
}

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento..."
}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [10]:
df = pd.DataFrame(documents)
df

,Título,Conteúdo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [11]:
model= "models/embedding-001"

In [29]:
def embed_fn(title, tex):
  return genai.embed_content(model=model,
                             content=sample_text, title= title,
                             task_type= "RETRIEVAL_DOCUMENT")["embedding"]

In [30]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Título"], row["Conteúdo"]), axis = 1)
df

,Título,Conteúdo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[0.03131934, -0.042252395, -0.049065363, 0.005..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.042142246, -0.05814502, -0.03486958, -0.037..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[0.050355952, -0.023072593, -0.023389122, -0.0..."


In [39]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                             content=consulta,
                             task_type= "RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteúdo"]

In [40]:
consulta = "Como ajustar a temperatura no controle climático?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema...


In [48]:
generation_config = {
    "temperature": 0.5,
    "candidate_count": 1
}

In [49]:
prompt = f"Reescreva o texto da resposta, de forma mais creativa e descontraída, mas sem adicionar informações fora do contexto do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro")
response = model_2.generate_content(prompt)
print(response.text)

Imagine seu Googlecar como um spa sobre rodas! Com o sistema de controle climático, você pode transformar o interior em um oásis aconchegante ou refrescante. É como ter um atendente pessoal ajustando a temperatura e o fluxo de ar, deixando você relaxado e focado na viagem. Basta um toque na tela para personalizar seu ambiente, criando o clima perfeito para qualquer aventura!
